In [52]:

from dotenv import load_dotenv
load_dotenv()
import os
import sys
sys.path.append(
    os.environ.get('WORK_DIR')) #type: ignore
sys.path.append(
    os.environ.get('DBASE_DIR')) #type: ignore
from dbase.DataAPI.ThetaData import * #type: ignore
from dbase.database.SQLHelpers import * #type: ignore
import pandas as pd
from data import HistoricTradeDataHandler
from event import *
from queue import Queue
from trade.backtester_.backtester_ import PTDataset, PTBacktester
import pandas_ta as ta
from trade.assets.Stock import Stock
from trade.backtester_.utils.WalkForwardUtils import prev_monday 
from trade.backtester_.strats import BBandsTrend2
from trade.backtester_.strats import MAStrat
import yfinance as yf
from datetime import datetime
from backtest import OptionSignalBacktest
from copy import deepcopy
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:

def create_datasate(stocks: list, start: str,interval: str, engine: str = 'yf', timewidth = None, timeframe = None, end: str = datetime.today(), return_object = False ):
    dataset = []
    if engine.lower() == 'yf':
        for stock in stocks:
            start = prev_monday(start)
            data2 = yf.download(stock, start = start, end = end, interval=interval, progress = False)

            dataset.append(PTDataset(stock, data2))
    else:
        for stk in stocks:
            stock = Stock(stk)
            data = stock.spot(ts = True, ts_start = '2018-01-01')
            data.rename(columns = {x:x.capitalize() for x in data.columns}, inplace= True)
            data['Timestamp'] = pd.to_datetime(data['Timestamp'], format = '%Y-%m-%d')
            data2 = data.set_index('Timestamp')
            data2 = data2.asfreq('W', method = 'ffill')
            data2 = data2.fillna(0)
            data2['Next_Day_Open'] = data2.Open.shift(-1)
            data2['EMA'] = ta.ma('ema', data2.Close, length = 21).fillna(0)
            dataset.append(PTDataset(stk, data2))
    return dataset if return_object else data2


In [3]:
start, end, interval = '2023-05-29', '2024-05-28','1d'
STOCKS = ['AAPL', 'MSFT','GOOGL', 'AMD', 'AMZN']
dataset = create_datasate(STOCKS, start, interval,end = end , return_object=True)
MAStrat.start_date = pd.to_datetime('1994-03-22')
tt = PTBacktester(dataset, MAStrat, cash =1000, commission = 0.0035)
stats = tt.run()
trades = tt.trades()
shorts = tt.trades()[tt.trades()['Size'] < 0]
shorts

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Tag,Duration,Ticker
0,-8,37,38,121.234194,121.360001,-1.006456,-0.001038,2023-07-24,2023-07-25,None,1 days,GOOGL
0,-9,37,38,110.232833,111.139999,-8.164500,-0.008230,2023-07-24,2023-07-25,None,1 days,AMD
1,-8,39,40,111.518319,111.790001,-2.173458,-0.002436,2023-07-26,2023-07-27,None,1 days,AMD
0,-2,40,41,339.288331,333.670013,11.236635,0.016559,2023-07-27,2023-07-28,None,1 days,MSFT
2,-8,41,42,112.983167,114.160004,-9.414691,-0.010416,2023-07-28,2023-07-31,None,3 days,AMD
...,...,...,...,...,...,...,...,...,...,...,...,...
55,-4,238,239,152.594050,154.289993,-6.783774,-0.011114,2024-05-09,2024-05-10,None,1 days,AMD
56,-4,240,241,150.750519,150.429993,1.282104,0.002126,2024-05-13,2024-05-14,None,1 days,AMD
57,-4,242,243,155.045431,160.919998,-23.498267,-0.037889,2024-05-15,2024-05-16,None,1 days,AMD
59,-4,249,249,160.845069,161.410004,-2.259740,-0.003512,2024-05-24,2024-05-24,None,0 days,AMD


In [36]:
trades_ = trades[:20]
trades_

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Tag,Duration,Ticker
0,-8,37,38,121.234194,121.360001,-1.006456,-0.001038,2023-07-24,2023-07-25,None,1 days,GOOGL
0,-9,37,38,110.232833,111.139999,-8.164500,-0.008230,2023-07-24,2023-07-25,None,1 days,AMD
1,7,39,40,130.525252,131.669998,8.013221,0.008770,2023-07-26,2023-07-27,None,1 days,GOOGL
1,-8,39,40,111.518319,111.790001,-2.173458,-0.002436,2023-07-26,2023-07-27,None,1 days,AMD
0,-2,40,41,339.288331,333.670013,11.236635,0.016559,2023-07-27,2023-07-28,None,1 days,MSFT
2,-8,41,42,112.983167,114.160004,-9.414691,-0.010416,2023-07-28,2023-07-31,None,3 days,AMD
0,7,42,43,133.666197,133.550003,-0.813357,-0.000869,2023-07-31,2023-08-01,None,1 days,AMZN
1,-3,43,44,334.016837,333.630005,1.160498,0.001158,2023-08-01,2023-08-02,None,1 days,MSFT
3,-8,43,44,113.860092,119.489998,-45.039246,-0.049446,2023-08-01,2023-08-02,None,1 days,AMD
2,-3,45,46,324.859000,331.880005,-21.063015,-0.021612,2023-08-03,2023-08-04,None,1 days,MSFT


In [8]:
import json
with open('profitable_weights.json', 'r') as f:
    weights = json.load(f)
trades_ = pd.read_csv('profitable_trades.csv').iloc[:, 1:]
trades_, weights

(     Size  EntryBar  ExitBar  EntryPrice   ExitPrice           PnL  ReturnPct  \
 0    27.0     504.0    568.0  336.262811  314.029999   -600.285939  -0.066117   
 1     9.0     504.0    571.0   52.904521   51.693333    -10.900697  -0.022894   
 2    37.0     504.0    584.0  192.240502  170.369995   -809.208768  -0.113766   
 3    21.0     504.0    584.0  130.695846  120.629997   -211.382813  -0.077017   
 4    42.0     504.0    585.0  279.795877  210.600006  -2906.226592  -0.247308   
 5    33.0     504.0    672.0  120.480208  132.740005    404.573332   0.101758   
 6    73.0     504.0    753.0  116.506348  161.250000   3266.286562   0.384045   
 7    64.0     504.0    753.0   87.043588  160.820007   4721.690846   0.847580   
 8    21.0     504.0    753.0  463.717356  911.770020   9409.105932   0.966219   
 9   422.0     504.0    753.0   42.282471  123.470001  34261.137831   1.920123   
 10    8.0     518.0    560.0  326.207752  306.160004   -160.381990  -0.061457   
 11   14.0     5

In [10]:

from EventDriven.riskmanager_async import RiskManager, close_cache, spot_cache
from pandas.tseries.offsets import BDay

rm = RiskManager(None, None, 1000000)
tick = 'GOOGL'
date = '2023-07-24'
start = (pd.to_datetime(date) - BDay(30)).strftime('%Y-%m-%d')
right = 'P'
order_settings = {'type': 'spread',
 'specifics': [{'direction': 'long',
   'rel_strike': 1.0,
   'dte': 365,
   'moneyness_width': 0.1},
  {'direction': 'short',
   'rel_strike': 0.85,
   'dte': 365,
   'moneyness_width': 0.1}],
 'name': 'vertical_spread'}

order = await rm.OrderPicker.get_order(tick, date, right, 5, order_settings)
order

                 long              short  close
10  GOOGL20240621P115  GOOGL20240621P100    5.0


{'result': 'SUCCESSFUL',
 'data': {'long': ['GOOGL20240621P115'],
  'short': ['GOOGL20240621P100'],
  'trade_id': '&L:GOOGL20240621P115&S:GOOGL20240621P100',
  'close': 5.0}}

In [ ]:
trades_ = pd.read_csv('profitable_trades.csv').iloc[:, 1:]
trades_

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,Ticker
0,27.0,504.0,568.0,336.262811,314.029999,-600.285939,-0.066117,2023-07-05,2023-10-04,91 days,MSFT
1,9.0,504.0,571.0,52.904521,51.693333,-10.900697,-0.022894,2023-07-05,2023-10-09,96 days,WMT
2,37.0,504.0,584.0,192.240502,170.369995,-809.208768,-0.113766,2023-07-05,2023-10-26,113 days,AAPL
3,21.0,504.0,584.0,130.695846,120.629997,-211.382813,-0.077017,2023-07-05,2023-10-26,113 days,AMZN
4,42.0,504.0,585.0,279.795877,210.600006,-2906.226592,-0.247308,2023-07-05,2023-10-27,114 days,TSLA
5,33.0,504.0,672.0,120.480208,132.740005,404.573332,0.101758,2023-07-05,2024-03-05,244 days,GOOG
6,73.0,504.0,753.0,116.506348,161.250000,3266.286562,0.384045,2023-07-05,2024-07-01,362 days,AMD
7,64.0,504.0,753.0,87.043588,160.820007,4721.690846,0.847580,2023-07-05,2024-07-01,362 days,AVGO
8,21.0,504.0,753.0,463.717356,911.770020,9409.105932,0.966219,2023-07-05,2024-07-01,362 days,LLY
9,422.0,504.0,753.0,42.282471,123.470001,34261.137831,1.920123,2023-07-05,2024-07-01,362 days,NVDA


In [157]:
w_map = {x: w  * 0.75 for x, w in weights.items()}
w_map

{'NVDA': 0.1305267587179231,
 'TSLA': 0.08662982076175331,
 'AMD': 0.06257152462300072,
 'LLY': 0.07318154887291858,
 'AAPL': 0.05282616482315278,
 'MSFT': 0.06813860424894518,
 'AVGO': 0.04131986733726825,
 'SBUX': 0.05415882705457591,
 'GOOG': 0.029099701121003622,
 'BAC': 0.033103416822703416,
 'AMZN': 0.020717547888259906,
 'MU': 0.014008829744854458,
 'QCOM': 0.013784459798162976,
 'DIS': 0.02244852689338821,
 'HD': 0.019189134465172068,
 'BA': 0.010770219379755886,
 'PFE': 0.010025047447161551,
 'INTC': 0.0037500000000000007,
 'WMT': 0.0037500000000000007}

In [156]:
0.009*100_000

899.9999999999999

In [159]:
#Backtest class 
evb_backtest = OptionSignalBacktest(trades_)

In [158]:
evb_backtest.portfolio.weight_map = w_map
evb_backtest.portfolio.weight_map
evb_backtest.portfolio.order_settings = {'type': 'naked',
 'specifics': [{'direction': 'long',
   'rel_strike': .900,
   'dte': 365,
   'moneyness_width': 0.15},
   {'direction': 'short',
   'rel_strike': .80,
   'dte': 365,
   'moneyness_width': 0.15}],

 'name': 'vertical_spread'}
evb_backtest.portfolio.max_price = int(10)

In [120]:
signals = evb_backtest.bars.signal_df
signals

,Date,MSFT,WMT,AAPL,AMZN,TSLA,GOOG,AMD,AVGO,LLY,NVDA,HD,QCOM,BA,INTC,MU,SBUX,BAC,DIS
0,2023-07-05,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0
1,2023-07-06,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2023-07-07,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2023-07-08,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2023-07-09,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358,2024-06-27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
359,2024-06-28,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
360,2024-06-29,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
361,2024-06-30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [121]:
signals_df = deepcopy(signals).set_index('Date')
signals_df[signals_df!=-1].sum().sum()

31.0

In [122]:
import cProfile
import pstats
import io

profiler = cProfile.Profile()
profiler.enable()
#run backtest

evb_backtest.run()
profiler.disable()
stream = io.StringIO()
stats = pstats.Stats(profiler, stream=stream).sort_stats('cumulative')

        Date  MSFT  WMT  AAPL  AMZN  TSLA  GOOG  AMD  AVGO  LLY  NVDA  HD  \
0 2023-07-05     1    1     1     1     1     1    1     1    1     1   0   

   QCOM  BA  INTC  MU  SBUX  BAC  DIS  
0     0   0     0   0     0    0    0  
Processing event: MARKET
Processing event: SIGNAL
                long             short  close
72  MSFT20240621C350  MSFT20240621C370  9.375
Processing event: SIGNAL
Moneyness too tight
Processing event: SIGNAL
                long             short  close
69  AAPL20240621C205  AAPL20240621C235  9.925
Processing event: SIGNAL
               long             short  close
3  AMZN20240621C125  AMZN20240621C145   9.75
Processing event: SIGNAL
                 long                short  close
161  TSLA20240621C290  TSLA20240621C316.67   10.0
Processing event: SIGNAL
                long             short  close
40  GOOG20240621C125  GOOG20240621C150  9.975
Processing event: SIGNAL
              long            short  close
3  AMD20240621C110  AMD20240621C135 

In [138]:
evb_backtest.portfolio.moneyness_count

6

In [14]:
stats.print_stats(30)
print(stream.getvalue())
with open('bactest_data.txt', 'w') as f:
  stream.seek(0)
  f.write(stream.read())
  f.flush()

         291176502 function calls (264139354 primitive calls) in 826.490 seconds

   Ordered by: cumulative time
   List reduced from 2981 to 30 due to restriction <30>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000  826.504  413.252 /Users/chiemelienwanisobi/miniconda3/envs/openbb/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3541(run_code)
        2    0.000    0.000  826.504  413.252 {built-in method builtins.exec}
        1    0.000    0.000  826.504  826.504 /var/folders/j0/80hkbygd4lb27h9mw76gqzpw0000gn/T/ipykernel_43067/2161208997.py:1(<module>)
        1    0.051    0.051  826.504  826.504 /Users/chiemelienwanisobi/cloned_repos/QuantTools/EventDriven/backtest.py:37(run)
       62    0.001    0.000  664.107   10.711 /Users/chiemelienwanisobi/cloned_repos/QuantTools/EventDriven/portfolio.py:270(update_signal)
       62    0.012    0.000  664.102   10.711 /Users/chiemelienwanisobi/cloned_repos/QuantTools/EventDr

In [23]:
evb_backtest.trades

,Ticker,PnL,EntryPrice,ExitPrice,ReturnPct,Quantity,EntryTime,ExitTime,Duration,Positions
0,MSFT,-4021.729957,485.231603,270.536235,-44.245957,18.732262,2023-07-05,2023-10-04,91,&L:MSFT20240621C355&S:MSFT20240621C365
1,AAPL,-5433.192644,482.278316,110.088399,-77.173264,14.597904,2023-07-05,2023-10-26,113,&L:AAPL20240621C200&S:AAPL20240621C210
2,AMZN,-570.587808,489.369516,388.155308,-20.682573,5.637428,2023-07-05,2023-10-26,113,&L:AMZN20240621C130&S:AMZN20240621C140
3,TSLA,-7212.238189,498.356369,187.716644,-62.332849,23.217372,2023-07-05,2023-10-27,114,&L:TSLA20240621C293.33&S:TSLA20240621C306.67
4,GOOG,570.885387,497.681533,570.882151,14.708325,7.798915,2023-07-05,2024-03-05,244,&L:GOOG20240621C127.5&S:GOOG20240621C137.5
5,AMD,17081.493376,489.487683,1492.731388,204.957906,17.026265,2023-07-05,2024-07-01,362,&L:AMD20240621C125&S:AMD20240621C140
6,BA,-399.175537,447.858690,322.933226,-27.893947,3.195310,2023-08-03,2023-09-11,39,&L:BA20240621C240&S:BA20240621C250
7,INTC,-217.222380,473.481778,266.664120,-43.680173,1.050309,2023-09-12,2023-10-24,42,&L:INTC20240621C37&S:INTC20240621C50
8,MU,-540.438854,447.419068,317.373543,-29.065709,4.155767,2023-09-14,2023-10-27,43,&L:MU20240621C70&S:MU20240621C80
9,AAPL,1575.913515,498.242601,609.895259,22.409296,14.114429,2023-11-03,2024-02-16,105,&L:AAPL20240920C180&S:AAPL20240920C190


In [71]:
evb_backtest.trades

,Ticker,PnL,EntryPrice,ExitPrice,ReturnPct,Quantity,EntryTime,ExitTime,Duration,Positions
0,AAPL,-4968.394740,1429.343912,422.402061,-70.447836,4.934143,2023-07-05,2023-10-26,113,&L:AAPL20240621C205
1,AMZN,-1173.275766,1427.543951,821.227811,-42.472678,1.935089,2023-07-05,2023-10-26,113,&L:AMZN20240621C142.5
2,GOOG,-117.577005,1375.666548,1333.999011,-3.028898,2.821789,2023-07-05,2024-03-05,244,&L:GOOG20240621C127.5
3,INTC,-250.914264,662.387877,329.470486,-50.260188,0.753683,2023-09-12,2023-10-24,42,&L:INTC20240621C37
4,MU,-787.556414,1140.685126,659.332998,-42.198510,1.636134,2023-09-14,2023-10-27,43,&L:MU20240621C70
5,AAPL,-565.461450,1297.542183,1193.261653,-8.036774,5.422503,2023-11-03,2024-02-16,105,&L:AAPL20240920C190
6,QCOM,8189.885614,1336.316225,7295.918927,445.972487,1.374233,2023-11-08,2024-07-01,236,&L:QCOM20250117C130
7,SBUX,-2415.174076,1253.764063,834.097005,-33.472570,5.754977,2023-11-08,2023-12-05,27,&L:SBUX20250117C105
8,AMZN,4764.704675,1367.468556,3732.663460,172.961557,2.014508,2023-11-14,2024-07-01,230,&L:AMZN20240920C160
9,INTC,140.393101,685.261683,878.123847,28.144309,0.727945,2023-11-14,2024-03-15,122,&L:INTC20240920C37


In [84]:
evb_backtest.trades

,Ticker,PnL,EntryPrice,ExitPrice,ReturnPct,Quantity,EntryTime,ExitTime,Duration,Positions
0,MSFT,-5788.966279,778.989421,281.981108,-63.801677,11.647625,2023-07-05,2023-10-04,91,&L:MSFT20240621C440
1,AAPL,-5696.346495,545.588925,104.825974,-80.786638,12.923832,2023-07-05,2023-10-26,113,&L:AAPL20240621C230
2,AMZN,-1312.193840,956.742858,501.901770,-47.540578,2.884950,2023-07-05,2023-10-26,113,&L:AMZN20240621C155
3,GOOG,-1296.358956,702.161320,467.444434,-33.427772,5.523075,2023-07-05,2024-03-05,244,&L:GOOG20240621C145
4,AMD,-2992.452021,955.329169,612.786171,-35.856018,8.735989,2023-07-05,2024-07-01,362,&L:AMD20240621C155
5,QCOM,-1029.520276,752.894353,330.064102,-56.160635,2.434831,2023-08-01,2023-08-09,8,&L:QCOM20240621C160
6,BA,-822.129255,987.848814,420.634843,-57.419107,1.449416,2023-08-03,2023-09-11,39,&L:BA20240621C290
7,INTC,-318.025324,259.428442,93.827476,-63.833003,1.920432,2023-09-12,2023-10-24,42,&L:INTC20240621C47
8,MU,-1022.999527,530.235876,238.985683,-54.928421,3.512442,2023-09-14,2023-10-27,43,&L:MU20240621C85
9,AAPL,-2090.634855,591.821113,415.458805,-29.799935,11.854204,2023-11-03,2024-02-16,105,&L:AAPL20240920C210


In [33]:
trades_

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,Ticker
0,27.0,504.0,568.0,336.262811,314.029999,-600.285939,-0.066117,2023-07-05,2023-10-04,91 days,MSFT
1,9.0,504.0,571.0,52.904521,51.693333,-10.900697,-0.022894,2023-07-05,2023-10-09,96 days,WMT
2,37.0,504.0,584.0,192.240502,170.369995,-809.208768,-0.113766,2023-07-05,2023-10-26,113 days,AAPL
3,21.0,504.0,584.0,130.695846,120.629997,-211.382813,-0.077017,2023-07-05,2023-10-26,113 days,AMZN
4,42.0,504.0,585.0,279.795877,210.600006,-2906.226592,-0.247308,2023-07-05,2023-10-27,114 days,TSLA
5,33.0,504.0,672.0,120.480208,132.740005,404.573332,0.101758,2023-07-05,2024-03-05,244 days,GOOG
6,73.0,504.0,753.0,116.506348,161.250000,3266.286562,0.384045,2023-07-05,2024-07-01,362 days,AMD
7,64.0,504.0,753.0,87.043588,160.820007,4721.690846,0.847580,2023-07-05,2024-07-01,362 days,AVGO
8,21.0,504.0,753.0,463.717356,911.770020,9409.105932,0.966219,2023-07-05,2024-07-01,362 days,LLY
9,422.0,504.0,753.0,42.282471,123.470001,34261.137831,1.920123,2023-07-05,2024-07-01,362 days,NVDA


In [123]:
from copy import deepcopy
test_data  =deepcopy(evb_backtest.portfolio.options_data['AAPL20240621C200'] - evb_backtest.portfolio.options_data['AAPL20240621C210'])
test_data[(test_data.index == '2023-07-05') | (test_data.index == '2023-10-26')]

KeyError: 'AAPL20240621C200'

In [137]:
weights

{'NVDA': 0.17403567829056413,
 'TSLA': 0.11550642768233775,
 'AMD': 0.0834286994973343,
 'LLY': 0.09757539849722477,
 'AAPL': 0.07043488643087037,
 'MSFT': 0.0908514723319269,
 'AVGO': 0.055093156449691004,
 'SBUX': 0.07221176940610122,
 'GOOG': 0.038799601494671494,
 'BAC': 0.04413788909693789,
 'AMZN': 0.027623397184346542,
 'MU': 0.018678439659805944,
 'QCOM': 0.018379279730883966,
 'DIS': 0.02993136919118428,
 'HD': 0.025585512620229425,
 'BA': 0.014360292506341181,
 'PFE': 0.013366729929548736,
 'INTC': 0.005000000000000001,
 'WMT': 0.005000000000000001}

In [136]:
pd.set_option('display.max_rows', 270)
pd.set_option('display.max_columns', 270)
avail = 9100
boght = 9085
market_value = 9805 + (avail - boght)
pd.DataFrame(evb_backtest.portfolio.all_holdings)#.set_index('datetime').plot(y = 'AMD')

,MSFT,WMT,AAPL,AMZN,TSLA,GOOG,AMD,AVGO,LLY,NVDA,HD,QCOM,BA,INTC,MU,SBUX,BAC,DIS,datetime,cash,commission,total
0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2023-07-05,100000.000000,0.000000,100000.000000
1,9085.147233,0.0,7043.488643,2762.339718,11550.642768,3879.960149,8342.869950,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2023-07-05,99573.852586,0.121451,142238.301049
2,9303.190767,0.0,7149.939353,2656.095883,10973.110630,3520.164346,8300.520356,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2023-07-06,99573.852586,0.121451,141476.873921
3,9327.417826,0.0,7008.005073,2691.510495,11290.753306,3461.819081,8406.394340,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2023-07-07,99573.852586,0.121451,141759.752707
4,8140.291921,0.0,6528.976878,2578.183737,10828.727595,2654.709576,8258.170762,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2023-07-10,99573.852586,0.121451,138562.913056
5,8237.200158,0.0,6369.300813,2670.261728,10713.221168,2888.090638,7940.548810,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2023-07-11,99573.852586,0.121451,138392.475901
6,8503.697810,0.0,6653.169373,2755.256796,10973.110630,3374.301183,8385.219543,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2023-07-12,99573.852586,0.121451,140218.607922
7,9085.147233,0.0,6812.845438,2939.412777,11435.136341,3918.856993,8660.491902,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2023-07-13,99573.852586,0.121451,142425.743271
8,8745.968403,0.0,6883.812578,3010.242001,11666.149196,3675.751721,8660.491902,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2023-07-14,99573.852586,0.121451,142216.268387
9,9545.461360,0.0,7504.775053,2896.915243,12590.200617,4278.652796,9020.463448,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2023-07-17,99573.852586,0.121451,145410.321104


In [146]:
0.1, 0.1, 0.1, 0.1
trades_[trades_['Ticker'] == 'TSLA']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,Ticker
4,42.0,504.0,585.0,279.795877,210.600006,-2906.226592,-0.247308,2023-07-05,2023-10-27,114 days,TSLA
22,37.0,599.0,638.0,240.328221,215.100006,-933.443933,-0.104974,2023-11-16,2024-01-16,61 days,TSLA


In [150]:
(evb_backtest.trades)

,Ticker,PnL,EntryPrice,ExitPrice,ReturnPct,Quantity,EntryTime,ExitTime,Duration,Positions
0,MSFT,-3439.157496,935.689267,580.801221,-37.927981,9.690824,2023-07-05,2023-10-04,91,&L:MSFT20240621C350&S:MSFT20240621C370
1,AAPL,-4647.365902,993.984027,339.122368,-65.882513,7.096714,2023-07-05,2023-10-26,113,&L:AAPL20240621C205&S:AAPL20240621C235
2,AMZN,-573.710002,974.733142,772.235510,-20.774674,2.833169,2023-07-05,2023-10-26,113,&L:AMZN20240621C125&S:AMZN20240621C145
3,TSLA,-7317.067788,998.258574,364.781465,-63.458219,11.550643,2023-07-05,2023-10-27,114,&L:TSLA20240621C290&S:TSLA20240621C316.67
4,GOOG,709.401108,996.089336,1178.469458,18.309615,3.889684,2023-07-05,2024-03-05,244,&L:GOOG20240621C125&S:GOOG20240621C150
5,AMD,12948.895694,983.071570,2511.881480,155.513592,8.469919,2023-07-05,2024-07-01,362,&L:AMD20240621C110&S:AMD20240621C135
6,QCOM,-702.845564,876.991636,539.989049,-38.427115,2.085579,2023-08-01,2023-08-09,8,&L:QCOM20240621C135&S:QCOM20240621C160
7,BA,-382.529321,961.313667,703.778435,-26.789927,1.485348,2023-08-03,2023-09-11,39,&L:BA20240621C230&S:BA20240621C250
8,INTC,-250.757931,395.537755,196.065779,-50.430578,1.257109,2023-09-12,2023-10-24,42,&L:INTC20240621C40&S:INTC20240621C55
9,MU,-573.765282,609.060371,420.848406,-30.902021,3.048506,2023-09-14,2023-10-27,43,&L:MU20240621C70&S:MU20240621C85


In [34]:
evb_backtest.portfolio.options_data['GOOGL20240621P115'] - evb_backtest.portfolio.options_data['GOOGL20240621P100']

,Open,High,Low,Close,Volume,Bid_size,CloseBid,Ask_size,CloseAsk,Midpoint,Weighted_midpoint
Datetime,,,,,,,,,,,
2023-07-24,4.16,4.13,4.05,4.07,-9,-134,5.85,38,4.15,5.000,5.381509
2023-07-24,4.16,4.13,4.05,4.07,-9,-134,5.85,38,4.15,5.000,5.381509
2023-07-25,4.25,4.25,3.93,3.91,-3198,2,3.30,27,4.35,3.825,3.857051
2023-07-25,4.25,4.25,3.93,3.91,-3198,2,3.30,27,4.35,3.825,3.857051
2023-07-26,2.88,3.12,2.88,3.12,-910,18,3.02,-191,3.05,3.035,2.938961
2023-07-26,2.88,3.12,2.88,3.12,-910,18,3.02,-191,3.05,3.035,2.938961
2023-07-27,2.88,2.85,2.70,2.85,-25,-23,3.07,14,3.10,3.085,3.114296
2023-07-27,2.88,2.85,2.70,2.85,-25,-23,3.07,14,3.10,3.085,3.114296
2023-07-28,2.66,2.69,2.65,2.64,174,62,1.10,6,2.77,1.935,1.839688


In [24]:
from trade.helpers.helper import generate_option_tick
import pandas as pd

tick = 'AAPL'
exp = '2024-03-12'
right = 'C'
strike = 145.0
option_tick = generate_option_tick(tick, right, exp, strike)
option_tick


'AAPL20240312000145C'

In [35]:
evb_backtest.portfolio.current_positions

{'GOOGL': {}, 'AMD': {}, 'MSFT': {}}

In [26]:
stats.print_stats(20)  # Show the top 20 functions by cumulative time
print(stream.getvalue())

         53676519 function calls (52386882 primitive calls) in 216.194 seconds

   Ordered by: cumulative time
   List reduced from 2937 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000  216.195  108.097 /Users/chiemelienwanisobi/miniconda3/envs/openbb/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3541(run_code)
        2    0.000    0.000  216.195  108.097 {built-in method builtins.exec}
        1    0.000    0.000  216.195  216.195 /var/folders/j0/80hkbygd4lb27h9mw76gqzpw0000gn/T/ipykernel_40825/2620833521.py:1(<module>)
        1    0.003    0.003  216.195  216.195 /Users/chiemelienwanisobi/cloned_repos/QuantTools/EventDriven/backtest.py:55(run)
       21    0.000    0.000  215.775   10.275 /Users/chiemelienwanisobi/cloned_repos/QuantTools/EventDriven/portfolio.py:270(update_signal)
       21    0.003    0.000  215.775   10.275 /Users/chiemelienwanisobi/cloned_repos/QuantTools/EventDriv

In [27]:
trades = evb_backtest.portfolio.get_trades()
print(trades.to_string())

KeyError: 'option'

In [80]:
#Get all holdings
holdings = evb_backtest.get_all_holdings()
print(holdings.to_string())

               AMD    MSFT   AMZN   GOOGL   AAPL      cash  commission     total
datetime                                                                        
2024-03-05     0.0     0.0    0.0     0.0    0.0  100000.0         0.0  100000.0
2024-03-05     0.0     0.0    0.0     0.0    0.0  100000.0         0.0  100000.0
2024-03-05  3680.0     0.0    0.0     0.0    0.0   96320.0         0.0  100000.0
2024-03-05  3680.0  1624.0    0.0     0.0    0.0   94696.0         0.0  100000.0
2024-03-05  3680.0  1624.0  416.0     0.0    0.0   94280.0         0.0  100000.0
2024-03-06  4596.0  1402.0    0.0     0.0    0.0   94280.0         0.0  100278.0
2024-03-07  4804.0  1770.0    0.0     0.0    0.0   94280.0         0.0  100854.0
2024-03-08  4630.0  1880.0    0.0     0.0    0.0   94280.0         0.0  100790.0
2024-03-11  3746.0     0.0  380.0     0.0    0.0   94280.0         0.0   98406.0
2024-03-12  3310.0  1990.0  378.0     0.0    0.0   94280.0         0.0   99958.0
2024-03-13  3178.0  2096.0  

In [36]:
positions = evb_backtest.get_all_positions()
print(positions.to_string())

                             AMD                   AAPL                   MSFT                   GOOGL
datetime                                                                                              
2024-03-04                  None                   None                   None                    None
2024-03-04                  None                   None                   None                    None
2024-03-05  AMD-20240816-270.0-C                   None  MSFT-20240816-480.0-C                    None
2024-03-06  AMD-20240816-270.0-C                   None  MSFT-20240816-480.0-C                    None
2024-03-07  AMD-20240816-270.0-C                   None  MSFT-20240816-480.0-C                    None
2024-03-08  AMD-20240816-270.0-C                   None  MSFT-20240816-480.0-C                    None
2024-03-11  AMD-20240816-270.0-C                   None  MSFT-20240816-480.0-C                    None
2024-03-12  AMD-20240816-270.0-C                   None  MSFT-20240816-48

In [64]:
evb_backtest.portfolio.all_positions

[{'AMD': {'quantity': 0.0, 'option': None},
  'MSFT': {'quantity': 0.0, 'option': None},
  'AMZN': {'quantity': 0.0, 'option': None},
  'GOOGL': {'quantity': 0.0, 'option': None},
  'AAPL': {'quantity': 0.0, 'option': None},
  'datetime': Timestamp('2024-03-04 00:00:00')},
 {'AMD': {'quantity': 0.0, 'option': None},
  'MSFT': {'quantity': 0.0, 'option': None},
  'AMZN': {'quantity': 0.0, 'option': None},
  'GOOGL': {'quantity': 0.0, 'option': None},
  'AAPL': {'quantity': 0.0, 'option': None},
  'datetime': Timestamp('2024-03-04 00:00:00')},
 {'AMD': {'quantity': 2,
   'option': 'AMD-20240920-240.0-C',
   'entry_price': 2185.0},
  'MSFT': {'quantity': 2,
   'option': 'MSFT-20250117-540.0-C',
   'entry_price': 778.0},
  'AMZN': {'quantity': 2,
   'option': 'AMZN-20250321-205.0-C',
   'entry_price': 0.0},
  'GOOGL': {'quantity': 0.0, 'option': None},
  'AAPL': {'quantity': 0.0, 'option': None},
  'datetime': Timestamp('2024-03-05 00:00:00')},
 {'AMD': {'quantity': 2,
   'option': 'AMD-20

In [7]:
from pprint import pprint

print('All positions:')
pprint(portfolio.all_positions)

print('\nCurrent holdings:')
pprint(portfolio.current_holdings)

print('\nCurrent positions:')
pprint(portfolio.current_positions)

All positions:
[{'AAPL': {'option': None, 'quantity': 0.0},
  'AMD': {'option': None, 'quantity': 0.0},
  'AMZN': {'option': None, 'quantity': 0.0},
  'GOOGL': {'option': None, 'quantity': 0.0},
  'MSFT': {'option': None, 'quantity': 0.0},
  'datetime': '20240226'},
 {'AAPL': {'option': None, 'quantity': 0.0},
  'AMD': {'option': None, 'quantity': 0.0},
  'AMZN': {'option': None, 'quantity': 0.0},
  'GOOGL': {'option': None, 'quantity': 0.0},
  'MSFT': {'option': None, 'quantity': 0.0},
  'datetime': Timestamp('2024-02-26 00:00:00')},
 {'AAPL': {'option': None, 'quantity': 0.0},
  'AMD': {'option': None, 'quantity': 0.0},
  'AMZN': {'option': None, 'quantity': 0.0},
  'GOOGL': {'option': None, 'quantity': 0.0},
  'MSFT': {'option': None, 'quantity': 0.0},
  'datetime': Timestamp('2024-02-26 00:00:00')}]

Current holdings:
{'AAPL': 0.0,
 'AMD': 0.0,
 'AMZN': 0.0,
 'GOOGL': 0.0,
 'MSFT': 0.0,
 'cash': 100000,
 'commission': 0.0,
 'total': 100000}

Current positions:
{'AAPL': {'option': N

# Test RiskManager 

In [13]:
from EventDriven.riskmanager import RiskManager
from dbase.DataAPI.ThetaData import list_contracts, retrieve_option_ohlc, is_theta_data_retrieval_successful #type: ignore
import datetime
import pandas as pd
import pandas_market_calendars as mcal
import unittest
import numpy as np
import pprint as pp

tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'NVDA', 'NFLX']


#generate date range 
nyse = mcal.get_calendar('NYSE')
year_ago_date = datetime.datetime.now() - datetime.timedelta(days=365)
schedule = nyse.schedule(start_date=year_ago_date, end_date=datetime.datetime.now())
date_range = mcal.date_range(schedule, frequency='1D')
dates = [date.strftime('%Y-%m-%d') for date in date_range]




class RiskManagerOperations(unittest.TestCase):
    def __init__(self):
        self.events = Queue(maxsize=0)  
        self.bars = HistoricTradeDataHandler(self.events, trades)
        self.risk_manager = RiskManager(bars=self.bars, events=self.events, initial_capital=1000000)
        
    def test_order_picker(self):
        ticker = np.random.choice(tickers)
        contract_date = np.random.choice(dates)
        contracts = list_contracts(ticker, pd.to_datetime(contract_date).strftime('%Y%m%d'))
        self.assertTrue(is_theta_data_retrieval_successful(contracts))
        
        contract = contracts.sample()
        print(contract)
        contract_right = contract['right'].values[0]
        contract_expiration = f"{contract['expiration'].values[0]}"
        contract_strike = float(contract['strike'].values[0])
        max_close = np.random.randint(1, 10)
        
        #order settings 
        moneyness_width = np.random.uniform(0.05, 0.1)
        rel_strike_long = np.random.uniform(1.05, 1.3) 
        rel_strike_short = np.random.uniform(0.7, 0.95)
        dte = np.random.randint(30, 365)
        
        order_settings = {
            'type': 'spread',
            'specifics': [
                {'direction': 'long', 'rel_strike': rel_strike_long, 'dte': dte, 'moneyness_width': moneyness_width},
                {'direction': 'short', 'rel_strike': rel_strike_short, 'dte': dte, 'moneyness_width': moneyness_width} 
            ],
            'name': 'vertical_spread'
        }
       
        try:
            self.order = self.risk_manager.OrderPicker.get_order(ticker, contract_expiration, contract_right, max_close, order_settings)
            self.assertIsInstance(self.order, dict)
            self.assertIsInstance(self.order['long'], list)
            self.assertIsInstance(self.order['short'], list)
            self.assertGreater(len(self.order['long']), 0)
            self.assertGreater(len(self.order['short']), 0)
            self.assertIsInstance(self.order['close'], float)
        except AssertionError as e:
            print(f"AssertionError: {e}")
            pp.pprint(contract)
            print(f"Ticker: {ticker}")
            print(f"Contract Date: {contract_date}")
            print(f"Contract Right: {contract_right}")
            print(f"Contract Expiration: {contract_expiration}")
            print(f"Contract Strike: {contract_strike}")
            print(f"Max Close: {max_close}")
            print(f"Order Settings: {order_settings}")
            raise
        except Exception as e:
            print(f"Exception: {e}")
            pp.pprint(contract)
            print(f"Ticker: {ticker}")
            print(f"Contract Date: {contract_date}")
            print(f"Contract Right: {contract_right}")
            print(f"Contract Expiration: {contract_expiration}, type: {type(contract_expiration)}")
            print(f"Contract Strike: {contract_strike}, type: {type(contract_strike)}")
            print(f"Max Close: {max_close}")
            print(f"Order Settings: {order_settings}")
            raise


ops = RiskManagerOperations()
ops.test_order_picker()

      root  expiration  strike right
3444  NVDA    20240301   270.0     C
Exception: Cannot set a DataFrame with multiple columns to the single column option_id
      root  expiration  strike right
3444  NVDA    20240301   270.0     C
Ticker: NVDA
Contract Date: 2024-02-09
Contract Right: C
Contract Expiration: 20240301, type: <class 'str'>
Contract Strike: 270.0, type: <class 'float'>
Max Close: 2
Order Settings: {'type': 'spread', 'specifics': [{'direction': 'long', 'rel_strike': 1.2305088940545148, 'dte': 284, 'moneyness_width': 0.08149699520032436}, {'direction': 'short', 'rel_strike': 0.9364378888363725, 'dte': 284, 'moneyness_width': 0.08149699520032436}], 'name': 'vertical_spread'}


ValueError: Cannot set a DataFrame with multiple columns to the single column option_id

In [15]:
from trade.assets.Stock import Stock
NVDA = Stock('NVDA', run_chain = False)

In [14]:
def generate_option_to_buy(underlier: Stock, contract_time):
        """
        Buy an option based on the underlier.
        """
        time = contract_time
        next_day_time = time + pd.DateOffset(days=1)
        print(time, next_day_time)
        option_spot = underlier.spot(ts=True, ts_start = time, ts_end = next_day_time)
        option_spot = option_spot.iloc[0] 
        stock_price = option_spot['open']#use open price as spot price on the assumption of making trades at start of day
        oom_benchmark = 0.1#10% out of the money 
        expiry_benchmark = time + pd.DateOffset(months=5)
        oom_price = stock_price * (1 + oom_benchmark)
        time_str = time.strftime("%Y%m%d")
        contracts = list_contracts(underlier.ticker, time_str)
        print(contracts)
        contracts = contracts[contracts['right'] == 'C'] 
        
        
        #Filter out contracts that are out of the money
        contracts = contracts[contracts['strike'] >= oom_price]
        
        print('comparing expiry')
        print(type(expiry_benchmark))
        print(type(contracts['expiration']))
        #filter out contracts that are  below the expiry benchmark
        contracts = contracts[pd.to_datetime(contracts['expiration'], format="%Y%m%d") >= expiry_benchmark]
        
        #select a random contract to buy
        contract = contracts.sample(n=1); 
        
        return contract
                

In [8]:
appl = Stock('AAPL')
c_time = pd.to_datetime('2024-05-06', format="%Y-%m-%d")
c_time_next = c_time + pd.DateOffset(days=1)
aapl_spot = appl.spot(ts=True, ts_start = c_time, ts_end = c_time_next)
aapl_spot.iloc[0]['open']


182.35000610351562

In [15]:
option = generate_option_to_buy(appl, c_time)
option

2024-05-06 00:00:00 2024-05-07 00:00:00
      root  expiration  strike right
0     AAPL    20241018   220.0     C
1     AAPL    20250919   215.0     C
2     AAPL    20241115   220.0     C
3     AAPL    20250321   220.0     C
4     AAPL    20250321   220.0     P
...    ...         ...     ...   ...
1060  AAPL    20240621   220.0     C
1061  AAPL    20240621   220.0     P
1062  AAPL    20240719   220.0     C
1063  AAPL    20250620   215.0     C
1064  AAPL    20240816   220.0     C

[1065 rows x 4 columns]
comparing expiry
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas.core.series.Series'>


,root,expiration,strike,right
456,AAPL,20250117,235.0,C
